In [1]:
import pandas as pd
import numpy as np
import astropy as ap
from tqdm import tqdm

from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [2]:
clusters=pd.read_pickle('/home/cz136/project/sa/data/cluster.pkl')
shapes=pd.read_pickle('/home/cz136/project/sa/data/shape.pkl')

In [3]:
def get_theta_for_obj(obj1,obj2):
    ra1,dec1=(obj1[('All', 'ra')],obj1[('All', 'dec')])
    ra2,dec2=(obj2[('All', 'ra')],obj2[('All', 'dec')])
    c1=SkyCoord(ra1*u.deg,dec1*u.deg)
    c2=SkyCoord(ra2*u.deg,dec2*u.deg)
    
    return(90-c1.position_angle(c2).degree)
    

In [4]:
def get_pa_for_obj(obj):
    e1=obj['All']['e1']
    e2=obj['All']['e1']
    
    α =0.5*np.arctan2(e2,e1)
    return(α)

In [5]:
def get_phi_for_cluster(cluster):
    center_id=cluster[('Alt', 'Alt1', 'ID_CENT')]
    center = shapes[shapes.index==center_id]
    
    mem_match_id=cluster.name
    members=shapes[shapes['All','MEM_MATCH_ID']==mem_match_id]
    
    pa=np.array([get_pa_for_obj(member[1]) for member in members.iterrows()]).flatten()
    theta= np.array([get_theta_for_obj(member[1],center) for member in members.iterrows()]).flatten()
    phi=pa-theta
    return(phi)    

In [6]:
phi_list=[]
for mem_matching_id in tqdm(clusters.index):
    phi_array=get_phi_for_cluster(clusters.loc[mem_matching_id])
    phi_list.append(phi_array)

100%|██████████| 7066/7066 [17:22<00:00,  6.78it/s]


In [15]:
np.save("/home/cz136/project/sa/data/phi_list",phi_list)

In [34]:
phi_1d[1]>=90 and phi_1d[1]<180

True

In [55]:
phi_1d=np.concatenate(phi_list)
phi_norm=np.zeros(len(phi_1d))

phi_1d=np.where(phi_1d<=0,-phi_1d,phi_1d)
phi_1d=np.where(phi_1d>=180,phi_1d-180,phi_1d)
phi_1d=np.where(phi_1d>=90,180-phi_1d,phi_1d)

#     if phi>=0 and phi<=90:
#         np.append(phi_norm,phi)
#     elif phi<=0:
#         np.append(phi_norm,-phi)
#     elif phi>=90 and phi<=180:
#         np.append(phi_norm,180-phi)
#     elif phi>=180:
#         np.append(phi_norm,phi-180)
#     else:
#         print ("Error")
#         print(phi)

array([57.29509629, 62.80005532, 69.57661541, ..., 70.63442668,
       71.34136119, 66.70996701])

In [56]:
pd.Series(phi_1d).describe()

count    394334.000000
mean         45.717587
std          26.406330
min           0.000014
25%          22.820737
50%          45.676874
75%          68.670759
max          89.999089
dtype: float64

In [44]:
np.append(phi_norm,phi_1d[1])

array([117.19994468])

In [ ]:
phi_norm